<a href="https://colab.research.google.com/github/hblacksmith/Clustering/blob/main/MSc_DTS_Sift_ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import Libraries


In [122]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

##Import Datasets


In [123]:
dataset = pd.read_csv('AppData.csv', encoding='ISO-8859–1')

In [124]:
dataset.head()

,Ref,Candidate,text,Cosine Similarity,Soft Cosine,Score
0,AppData1,3620013,"During my RAF cadet journey, I had to demonstr...",0.392927,0.61,REJECT
1,AppData2,3621201,As I have worked in my role at M&S for over 2 ...,0.424385,0.62,REJECT
2,AppData3,3624197,I drive my team with great confidence and a he...,0.388232,0.61,REJECT
3,AppData4,3624416,"Although I am not in a leadership position, I ...",0.315489,0.57,REJECT
4,AppData5,3625604,When I worked at Stansted Airport in the speci...,0.368704,0.60,REJECT


In [125]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ref                24 non-null     object 
 1   Candidate          24 non-null     int64  
 2   text               24 non-null     object 
 3   Cosine Similarity  24 non-null     float64
 4   Soft Cosine        24 non-null     float64
 5   Score              24 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 1.2+ KB


##Create TF-IDF Model

In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features = 1500)
X = tfidf.fit_transform(dataset['text'])
y = dataset.iloc[:, -1].values

##Split Dataset into Training and Test Sets

In [127]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

##Train the Naive Bayes model on the Training set

In [128]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

##Predict Test set Results

In [129]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[['REJECT' 'REJECT']
 ['REJECT' 'REJECT']
 ['REJECT' 'REJECT']
 ['REJECT' 'PASS']
 ['REJECT' 'REJECT']]


##Measuring Performance

In [130]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)


[[0 1]
 [0 4]]


0.8

In [131]:
#What the model got right
y_true = np.array([0,0,0,1,0])
y_pred = np.array([0,0,0,0,0])
true_positives = ((y_pred == y_true) & (y_pred == 1)).sum()
true_positives

0

In [132]:
true_negatives = ((y_pred == y_true) & (y_pred == 0)).sum()
true_negatives

4

In [133]:
#What the model got wrong
false_positives = ((y_pred != y_true) & (y_pred == 1)).sum()
false_positives

0

In [134]:
false_negatives = ((y_pred != y_true) & (y_pred == 0)).sum()
false_negatives

1

In [140]:
#Confusion Matrix
confusion = [[true_positives, false_positives], [false_negatives, true_negatives]]
confusion

[[0, 0], [1, 4]]

In [143]:
confusion = pd.DataFrame(confusion, columns=[0, 0], index=[1, 0])
confusion.index.name = r'pred \ truth'
confusion

,0,0
pred \ truth,,
1,0,0
0,4,4


In [135]:
prediction = pd.DataFrame(y_pred, columns=['predictions']).to_csv('prediction.csv')